In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from typing import Union

import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import io, img_as_ubyte
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from tqdm import trange, tqdm

from imblearn.under_sampling import RandomUnderSampler

from src.data.ucmerced_dataset import TripletDataModule
from src.measures import anmrr
from src.models.bovw import BoVWRetriever
from src.settings import RANDOM_WALKS_DIRECTORY, UC_MERCED_DATA_DIRECTORY, PATTERN_NET_DATA_DIRECTORY

In [11]:
image_size = 256
dm = TripletDataModule(UC_MERCED_DATA_DIRECTORY, image_size, 0.8, 100, augment=False, normalize=False, permute=True)
dm.setup(None)
train_dataset = dm.train_dataset
test_dataset = dm.val_dataset

model = BoVWRetriever(100, 10000)
model.fit(train_dataset)
model.eval(test_dataset)
# train_dataset = UcMercedDataset(TRAIN_DATA_DIRECTORY)
# test_dataset = UcMercedDataset(TEST_DATA_DIRECTORY)

Encoding as BOVW: 100%|██████████| 420/420 [00:00<00:00, 1253.73it/s]


0.6423447860156721

In [ ]:
x_train = np.empty(shape=(len(train_dataset), 256, 256, 3))
y_train = np.empty(shape=(len(train_dataset), ), dtype=np.int)

x_test = np.empty(shape=(len(test_dataset), 256, 256, 3))
y_test = np.empty(shape=(len(test_dataset), ), dtype=np.int)

for idx in trange(train_dataset.__len__()):
    item = train_dataset.__getitem__(idx)

    x_train[idx] = item['a']
    y_train[idx] = item['a_y']

for idx in trange(test_dataset.__len__()):
    item = test_dataset.__getitem__(idx)

    x_test[idx] = item['a']
    y_test[idx] = item['a_y']

In [ ]:
sift = cv2.SIFT_create()

In [ ]:
def get_descriptors(
        images: np.ndarray,
        labels: Union[np.ndarray, None] = None
):
    desc = []
    matching_labels = []

    for idx, img in tqdm(
            enumerate(images),
            desc="Calculating SIFT descriptors",
            total=images.shape[0]
    ):
        cv_img = img_as_ubyte(img)
        cv_img = cv2.cvtColor(cv_img, cv2.COLOR_RGB2GRAY)
        _, d = sift.detectAndCompute(cv_img, None)
        if d is not None:
            desc.append(d)

            if labels is not None:
                matching_labels.append(np.repeat(labels[idx], len(d)))

    if labels is not None:
        return desc, matching_labels
    else:
        return desc

def encode_as_bovw(x: np.ndarray, trained_kmeans: KMeans) -> np.ndarray:
    descriptors = get_descriptors(x)

    res = np.empty(shape=(x.shape[0], trained_kmeans.n_clusters))

    for idx, desc in tqdm(
            enumerate(descriptors),
            total=len(descriptors),
            desc="Encoding as BOVW"):
        words = trained_kmeans.predict(desc)
        bovw, _ = np.histogram(words, bins=range(trained_kmeans.n_clusters + 1))
        res[idx] = bovw / desc.shape[0]

    return res

In [ ]:
x_train_descriptors, y_train_descriptors = get_descriptors(x_train, y_train)
stacked_train_descriptors = np.vstack(x_train_descriptors)
stacked_train_labels = np.hstack(y_train_descriptors)

In [ ]:
np.unique(stacked_train_labels, return_counts=True)

In [ ]:
under_sampler = RandomUnderSampler(random_state=42)

resampled_train_descriptors, resampled_train_labels = under_sampler.fit_resample(stacked_train_descriptors, stacked_train_labels)

In [ ]:
np.unique(resampled_train_labels, return_counts=True)

In [ ]:
CLUSTERS = 100
SAMPLES_COUNT = 100000

In [ ]:
samples_ratio_for_kmeans = SAMPLES_COUNT / resampled_train_descriptors.shape[0]

_, descriptors_for_kmeans, _, labels_for_kmeans = train_test_split(
    resampled_train_descriptors,
    resampled_train_labels,
    test_size=samples_ratio_for_kmeans
)

In [ ]:
np.unique(labels_for_kmeans, return_counts=True)

In [ ]:
%%time

k_means = KMeans(n_clusters=CLUSTERS)

k_means.fit(descriptors_for_kmeans)


In [ ]:
x_test_encoded = encode_as_bovw(x_test, k_means)

In [ ]:
anmrr(x_test_encoded, y_test[:, None], euclidean_distances)

In [ ]:
n_neighbours = NearestNeighbors()
n_neighbours.fit(x_test_encoded)

In [ ]:
QUERY_IMAGE_IDX = 162

In [ ]:
query_image = x_test_encoded[QUERY_IMAGE_IDX].reshape(1, -1)
_, query_result_idx = n_neighbours.kneighbors(query_image, n_neighbors=11)

In [ ]:
for idx in query_result_idx.squeeze():
    io.imshow(x_test[idx])
    plt.title(test_dataset.label_name_mapping[y_test[idx]])
    plt.show()

In [ ]:
import imageio
from src.visualisation import random_walk

In [ ]:
random_walk(
    images=x_test,
    images_encoded=x_test_encoded,
    trained_n_neighbours=n_neighbours,
    search_range=15,
    starting_index=106,
    filename_prefix="uc_merced_classic_"
)